# Process the data (combine them together)

## Check JH's augmentation data

In [ ]:
import importlib
import functions

JH_aug_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/Tmp/JH_Pre_Augmentation/original_augmentation_data/JH_aug_2.mat"
JH_reshape_vol_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input/FA_17017030_AugVol_2.mat"
JH_reshape_pts_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output/FA_17017030_AugPts_2.mat"

pixel_space = [0.15, 0.15, 0.15]

importlib.reload(functions)

# JH_aug_volume, JH_aug_pts = functions.load_mat_data(JH_aug_mat_path)
# functions.show_pts(JH_aug_volume, JH_aug_pts, pixel_space)

JH_aug_zoomed_volume, JH_aug_zoomed_pts = functions.load_mat_data(JH_reshape_vol_mat_path, JH_reshape_pts_mat_path)
functions.show_pts(JH_aug_zoomed_volume, JH_aug_zoomed_pts, pixel_space)


In [ ]:
# Try up_sampling method

up_sampling_JH = functions.rescale_3d_volume(JH_aug_zoomed_volume, (512, 512, 90))
functions.show_pts(up_sampling_JH, JH_aug_zoomed_pts, pixel_space)

## Combination

In [35]:
import numpy as np
import importlib
import functions

def combine_data(x_base_path, y_base_path, store_base_path, data_tag):
    importlib.reload(functions)

    # X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
    # Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"

    x_dataset, y_dataset = functions.load_mat_data_dir(x_base_path, y_base_path)

    x_dataset = np.asarray(x_dataset)
    y_dataset = np.asarray(y_dataset)

    x_dataset_path = store_base_path + "X_" + data_tag + "_data"
    y_dataset_path = store_base_path + "Y_" + data_tag + "_data"

    np.save(x_dataset_path, x_dataset)
    np.save(y_dataset_path, y_dataset)


In [43]:
X_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Input"
Y_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Output"
X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"
X_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Input"
Y_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Output"
store_base_path = "data/"

combine_data(X_train_base_path, Y_train_base_path, store_base_path, "train")
combine_data(X_val_base_path, Y_val_base_path, store_base_path, "val")
combine_data(X_test_base_path, Y_test_base_path, store_base_path, "test")

In [3]:
import numpy as np
import functions

# Inspect Loaded Data

X_file_path = "data/X_train_data.npy"
Y_file_path = "data/Y_train_data.npy"

X_load = np.load(X_file_path, allow_pickle=True)
Y_load = np.load(Y_file_path, allow_pickle=True)

print(np.shape(X_load), np.shape(Y_load))
print(np.shape(X_load[1, :, :, :]), np.shape(Y_load[1, :, :]))

# functions.show_pts(X_load[1, :, :, :], Y_load[1, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[2, :, :, :], Y_load[2, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[3, :, :, :], Y_load[3, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[4, :, :, :], Y_load[4, :, :], [0.15, 0.15, 0.15])

(700, 170, 170, 30) (700, 4, 3)
(170, 170, 30) (4, 3)


In [ ]:
import Spartan.models as models

model_s1, model_s2 = models.spine_lateral_radiograph()

model_s1.summary()


In [2]:
from tensorflow import Tensor
from keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from keras.models import Model

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():

    inputs = Input(shape=(32, 32, 3))
    num_filters = 64

    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)

    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2

    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)

    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

model = create_res_net() # or create_plain_net()
model.summary()

KeyboardInterrupt: 